In [547]:
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt  # Biblioteca para gerar gráficos
import pandas as pd
from sklearn import metrics, model_selection
from scipy import stats
import math


In [548]:
data = np.genfromtxt('breastcancer.csv', delimiter=',')
np.random.seed(666)

## Questão 1

Considere o conjunto de dados disponível em `breastcancer.csv`, organizado
em 31 colunas, sendo as 30 primeiras colunas os atributos e a última coluna a
saída. Os 30 atributos coletados de exames médicos são usados no diagnóstico
do câncer de mama, sendo 1 a classe positiva e 0 a classe negativa. Maiores 
detalhes sobre os dados podem ser conferidos em 
https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset.

a) Considerando uma validação cruzada em 10 folds, avalie modelos de classicação binária nos dados em questão. Para tanto, use as abordagens abaixo:
- Regressão logística (treinado com GD ou SGD);
- Análise do discriminante Gaussiano;
- Naive Bayes Gaussiano;

In [549]:

class Set:
    def __init__(self, dataset, features, output):
        self.dataset = dataset
        self.features = features
        self.output = output

    def get_n(self):
        return self.dataset.shape[0]

    def get_x(self):
        return self.dataset[:, self.features]
    
    def get_x_apply(self, func):
        return func(self.dataset[:, self.features])

    def set_x(self, new_x):
        self.x = new_x

    def get_y(self):
        return self.dataset[:, self.output]

    def get_X(self, normal_fun=None):
        if (normal_fun):
            return np.c_[np.ones(self.get_n()), normal_fun(self.get_x())]
        else:
            return np.c_[np.ones(self.get_n()), self.get_x()]


In [550]:
def k_fold_split(array, k = int):
    """realiza o split dos dados em k-folds"""
    shuffled_data = np.random.permutation(array)
    folds = np.array_split(shuffled_data, k)
    return folds

def k_fold_train_test(folds):
    """retorna um vetor com as configurações de treino e teste definidas pelo k-fold split"""
    results = []
    for i, fold in enumerate(folds):
        train = np.vstack([x for j, x in enumerate(folds) if j != i])
        test = fold
        results.append((i, train, test))
    return results
    

folds = k_fold_split(data, 10)

In [551]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

def GD(X, y, learning_rate, epochs):
    # inicia valores
    n = y.shape[0]
    # w = np.random.randn(X.shape[1])
    w = np.zeros(X.shape[1])

    # atualiza pesos (w) em cada época
    for t in range(epochs):
        ei_sum = np.zeros(X.shape[1])
        for i in range(n):
            ei = y[i] - sigmoid(np.dot(w.T, X[i]))
            ei_sum += np.dot(ei, X[i])
        w = w + learning_rate * (ei_sum/n)
        # w = w + learning_rate * np.mean(np.dot(e, X))
    return w

def log_reg_GD_predict(train_set, test_set, learning_rate, epochs):
    features = np.arange(30)
    output = 30

    # pega colunas de treino
    train_set = Set(train_set, features, output)
    X_train = train_set.get_X(normal_fun=stats.zscore)
    y_train = train_set.get_y()

    # treina modelo
    w_train = GD(X_train, y_train, learning_rate, epochs)

    # pega colunas de teste
    test_set = Set(test_set, features, output)
    X_test = test_set.get_X(normal_fun=stats.zscore)
    y_test = test_set.get_y()

    # testa modelo
    y_pred = sigmoid(np.dot(w_train, X_test.T))
    y_pred = np.around(y_pred)

    return (y_test, y_pred)


In [552]:
def GDA(x, y):
    pCk_, mu_k_, sigma_k_ = [], [], []

    N = y.shape[0]
    for k in np.unique(y):
        # calcula p(Ck)
        n = y[y==k].shape[0]
        pCk = n/N
        pCk_.append(pCk)

        # calcula mu_k
        xk = x[y==k]
        mu_k = xk.mean()
        mu_k_.append(mu_k)
        
        # calcula sigma_k
        sigma_k = (xk - mu_k).T.dot(xk - mu_k) / (xk.shape[0] - 1)
        sigma_k_.append(sigma_k)

    return (pCk_, mu_k_, sigma_k_)

def GDA_predict(train_set, test_set):
    features = np.arange(30)
    output = 30

    # pega colunas de treino
    train_set = Set(train_set, features, output)
    x_train = train_set.get_x()
    y_train = train_set.get_y()

    # treina modelo
    pC, mu, sigma = GDA(x_train, y_train)
    
    # pega colunas de teste
    test_set = Set(test_set, features, output)
    x_test = test_set.get_x()
    y_test = test_set.get_y()

    y_pred = []
    for i, y in enumerate(y_test):
        y_probabilities = []
        # para cada classe, verifica as probabilidades
        for k, classification in enumerate(np.unique(y_test)):
            eq1 = np.log(pC[k])
            eq2 = (-0.5 * np.log(np.linalg.det(sigma[k])))
            aux = x_test[i] - mu[k]
            eq3 = (-0.5 * aux.T) @ (np.linalg.pinv(sigma[k]) @ aux)
            
            prob = eq1 + eq2 + eq3
            y_probabilities.append(prob)

        # escolhe a mais provável
        y_pred.append(np.argmax(y_probabilities))
        # print(y_probabilities)

    return (y_test, np.array(y_pred))


In [553]:
def naive_bayes_predict(train, test):
    features_i = np.arange(30)
    output_i = 30
    train = Set(train, features_i, output_i)
    test = Set(test, features_i, output_i)

    x, y, x_test, y_test = train.get_x_apply(stats.zscore), train.get_y(), test.get_x_apply(stats.zscore), test.get_y()

    # treina o modelo
    classes = np.unique(np.array(y, dtype='int64'))
    pC, mu, sigma = [], [], []
    for k in classes:
        pC.append(y[y==k].shape[0] / y.shape[0])
        mu.append(x[y==k].mean(axis=0))
        sigma.append(x[y==k].std(axis=0))

    # calcula probabilidades
    rows, D = range(x_test.shape[0]), x_test.shape[1]
    y_pred = []

    # para cada xi dentro do x de teste
    for i in rows:
        y_probabilities = []

        # calcula probabilidades das classes
        for k in classes:
            eq1 = np.log(pC[k])
            eq2 = -0.5 * np.sum(np.log(2 * np.pi * sigma[k]))
            eq3 = -0.5 * np.sum((x_test[i] - mu[k])**2 / sigma[k])
            prob = eq1 + eq2 + eq3
            y_probabilities.append(prob)

        # escolhe a mais provável
        y_pred.append(np.argmax(y_probabilities))

    return (y_test, np.array(y_pred))


b) Para cada modelo criado, reporte valor médio e desvio padrão das métricas de **acurácia**, **revocação**, **precisão** e **F1-score**

In [554]:
def is_true_positive(y, y_pred):
    return y_pred >= 1 and y >= 1

def is_false_positive(y, y_pred):
    return y_pred >= 1 and y <= 0

def is_true_negative(y, y_pred):
    return y_pred <= 0 and y <= 0

def is_false_negative(y, y_pred):
    return y_pred <= 0 and y >= 1

def get_prediction_matrix(y, y_pred):
    """ returns (tp, fp, tn, fn) """

    tp, fp, tn, fn = 0, 0, 0, 0
    for i, pred in enumerate(y_pred):
        tp += 1 if is_true_positive(y[i], pred) else 0
        fp += 1 if is_false_positive(y[i], pred) else 0
        tn += 1 if is_true_negative(y[i], pred) else 0
        fn += 1 if is_false_negative(y[i], pred) else 0
    return (tp, fp, tn, fn)

def accuracy(y, y_pred):
    tp, fp, tn, fn = get_prediction_matrix(y, y_pred)
    return (tp + tn) / (tp + fp + tn + fn)

def precision(y, y_pred):
    tp, fp, tn, fn = get_prediction_matrix(y, y_pred)
    return tp / (tp + fp)

def recall(y, y_pred):
    tp, fp, tn, fn = get_prediction_matrix(y, y_pred)
    return tp / (tp + fn)

def f1_score(y, y_pred):
    precision_ = precision(y, y_pred)
    recall_ = recall(y, y_pred)
    return 2 * (precision_ * recall_) / (precision_ + recall_)


In [555]:
models = [
    {
        'name': "Regressão logística (GD)",
        'function': log_reg_GD_predict,
        'args': [0.01, 100]
    },
    {
        'name': "Análise Discriminante Gaussiana",
        'function': GDA_predict
    },
    {
        'name': "Naive Bayes Gaussiana",
        'function': naive_bayes_predict
    }
]

n_folds = len(folds)

for model in models:
    accuracy_, precision_, recall_, f1_score_ = np.zeros(n_folds), np.zeros(n_folds), np.zeros(n_folds), np.zeros(n_folds)

    for i, train, test in k_fold_train_test(folds):
        y_test, y_pred = [], []
        if 'args' in model:
            y_test, y_pred = model['function'](train, test, *model['args'])
        else:
            y_test, y_pred = model['function'](train, test)
        
        accuracy_[i] = (accuracy(y_test, y_pred))
        precision_[i] = (precision(y_test, y_pred))
        recall_[i] = (recall(y_test, y_pred))
        f1_score_[i] = (f1_score(y_test, y_pred))

    print("%i-fold cross validation com %s" % (n_folds, model['name']))
    print("acurácia: %.8f +/- %.8f" % (accuracy_.mean(), accuracy_.std()))
    print("revocação: %.8f +/- %.8f" % (precision_.mean(), precision_.std()))
    print("precisão: %.8f +/- %.8f" % (recall_.mean(), recall_.std()))
    print("f1-score: %.8f +/- %.8f" % (f1_score_.mean(), f1_score_.std()))
    print("")
    

10-fold cross validation com Regressão logística (GD)
acurácia: 0.93668546 +/- 0.03071788
revocação: 0.94632035 +/- 0.05628245
precisão: 0.95434642 +/- 0.03264910
f1-score: 0.94870336 +/- 0.02630920

10-fold cross validation com Análise Discriminante Gaussiana
acurácia: 0.95598371 +/- 0.03719696
revocação: 0.96550712 +/- 0.05047300
precisão: 0.96412776 +/- 0.04898371
f1-score: 0.96310050 +/- 0.03161546

10-fold cross validation com Naive Bayes Gaussiana
acurácia: 0.92781955 +/- 0.03681396
revocação: 0.91590736 +/- 0.06625456
precisão: 0.97602470 +/- 0.02341703
f1-score: 0.94305233 +/- 0.03128240

